## 抓取前一天日期

In [1]:
import datetime
today = datetime.datetime.now()
yestoday = today + datetime.timedelta(days = -1)
print(yestoday.strftime('%Y%m%d'))

20161015


## 存取url

In [2]:
#抓日期
import datetime
today = datetime.datetime.now()
yestoday = today + datetime.timedelta(days = -1)
yestoday = yestoday.strftime('%Y%m%d')
file_datestring = yestoday
#把分類跟 url 組合在一起
import requests
from bs4 import BeautifulSoup
from urlparse import urljoin
import os
import datetime as dt
import time
urls = "" #裝要印出的所有url
setList = [] #裝不重複的url

#抓到昨天蘋果新聞的url
page_url = 'http://www.appledaily.com.tw/appledaily/archive/{}'.format(yestoday)

# print(page_url)
main_url ='http://www.appledaily.com.tw/appledaily'
res = requests.get(page_url)
soup = BeautifulSoup(res.text)

#包含所有種類的div
div = soup.select('.abdominis.clearmen')[0]

#抓取所有連結
lis = div.select('.nclns li')
for li in lis:
    href = li.select('a')[0]['href']
    category = href.split('/')[3]
    if(category=='enews'):
        category='entertainment'
    news_url = urljoin(main_url,href)+"\n" #urljoin會自動合併兩個網址
    
    #進行新聞的篩選
    if(category != 'supplement' and category != 'article' and category != 'forum' and category !='adcontent'):
        urls += category + news_url  #把新聞串在一起等著印出
time.sleep(0.5)
f = open('C:/Users/ytchen/Desktop/AppleNews/{}.txt'.format(file_datestring),'w')
f.write(urls.encode('utf-8')) #最後再一次寫出來
f.close()  

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


## 將檔案寫入file

In [4]:
#這個才是最正確的
#修改成一次爬好幾個月
import string
import datetime as dt
import requests
import time
from bs4 import BeautifulSoup
import json
import time

#全域變數
error_count = 0
error_url = ''

#抓日期
import datetime
today = datetime.datetime.now()
yestoday = today + datetime.timedelta(days = -1)
yestoday = yestoday.strftime('%Y%m%d')
file_datestring = yestoday

#準備抓資料
all_json_str = [] #存所有資訊的 json

#開起每天的url檔案
f = open('C:/Users/ytchen/Desktop/AppleNews/{}.txt'.format(file_datestring),'r')
for url_no in f.readlines():
#     print(url_no)
    try:
        category = url_no.split('http')[0]        #把類別抓出來
        url = 'http'+url_no.split('http')[1]      #重新組合url
        res = requests.get(url) #利用get發送請求
        soup = BeautifulSoup(res.text)
        file_date = file_datestring #日期當作檔案的名子
#         print(file_date)
        
        #抓標題跟副標題
        title = soup.select('#h1')[0].text
#         title2 = soup.select('#h2')[0].text            
#         title =''
#         titles = title1 + title2
#         title = '-'.join(titles.split())

        
        #抓人氣
        click = soup.select('.function_icon.clicked')[0].text
        click = click.split('(')[1].replace(')','')
        
        #抓內文
        article = ''
        div= soup.select('.articulum')[0]   #先把所有內文抓到
        ps = div.select('p')                #再把裡面所有的p標籤抓到
        for p in ps:                        #把每個p標籤一個一個抓出來印
            article += p.text.replace('\r','').replace('\n','')  #段落會隱藏換行符號\r，要取代掉，不然後面讀取json格式會只讀到最後一行
        
        #將資訊存成 json檔
        data  =  { 
            'comp':'apple',  #新聞來源
            'category':category, #類別
            'title':title, #標題
            'content':article, #內文
            'date':file_date,  #直接抓檔案的日期來用
            'url':url,
            'keyw':'',
            'click':click
        }
        
#         print(title)
        #一篇新聞一本字典
        all_json_str.append(data) 
        time.sleep(0.5)
        
    except:
        error_count +=1
        error_url += url+','
        continue 
        
            
# #檔案儲存
# f1 = open('C:/Users/ytchen/Desktop/AppleNews/json/{}.json'.format(file_datestring),'w')
# b =json.dumps(all_json_str) #再把 list 字典轉換成 json 並包起來
# f1.write(b)

#寫入檔案結束
f1.close()
f.close()


#log記錄
logString = ''
fileName_log = str(file_datestring)+'/'+'errorCount:'+str(error_count)+'\n'+'error_url:'+error_url+'\n'
#讀取
with open('C:/Users/ytchen/Desktop/AppleNews/log_content.txt','r') as f1:
    for logLine in f1.readlines():
        logString += logLine
#重新寫入
with open('C:/Users/ytchen/Desktop/AppleNews/log_content.txt','w') as f2:
    logString += fileName_log
    f2.write(logString)

## 將檔案寫入mongodb

In [3]:
import pymongo
from pymongo import MongoClient
import json
import datetime
today = datetime.datetime.now()
yestoday = today + datetime.timedelta(days = -1)
yestoday = yestoday.strftime('%Y%m%d')
file_datestring = yestoday

f = open('C:/Users/ytchen/Desktop/AppleNews/json/{}.json'.format(file_datestring),'r')
i = f.read()
json = json.loads(i)
print(len(json))
    
client = MongoClient() 
database = client["test"]
collection = database["news"]
collection.insert_many(
    json
)
f.close()
client.close()

96
